In [1]:
from __future__ import print_function, division, absolute_import

import argparse
import time

import numpy as np
seed = 7
np.random.seed(seed)

import os
import sys
sys.path.insert(0, os.path.abspath('..'))
os.chdir('../CoMET')

import evolutron.networks as nets
from evolutron.motifs import motif_extraction
from evolutron.tools import load_dataset, none2str, Handle
from evolutron.engine import DeepTrainer

from keras.utils.np_utils import to_categorical

Using Theano backend.
ERROR (theano.sandbox.cuda): ERROR: Not using GPU. Initialisation of device 1 failed:
initCnmem: cnmemInit call failed! Reason=CNMEM_STATUS_OUT_OF_MEMORY. numdev=1

Using gpu device 0: GeForce GTX 1080 (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5105)


In [2]:
# Data and model
data_id = 'dnabind'
padded = True
mode = 'family'

# Architecture
filters = 300
filter_length = 50
conv = 1
fc = 1

# Training
num_epochs = 200
batch_size = 50
optimizer = 'nadam'
rate = .005
validate = .2

In [3]:
dataset = load_dataset(data_id, padded=padded, codes=True)
x_data, y_data = dataset
if type(x_data) == np.ndarray:
    input_shape = x_data[0].shape
elif type(x_data) == list:
    input_shape = (None, x_data[0].shape[1])
else:
    raise TypeError('Something went wrong with the dataset type')

y_data = to_categorical(y_data)
output_dim = y_data.shape[1]

Dataset size: 6703


In [4]:
net_arch = nets.DeepCoFAM.from_options(input_shape,
                                               output_dim,
                                               n_conv_layers=conv,
                                               n_fc_layers=fc,
                                               n_filters=filters,
                                               filter_length=filter_length)
conv_net = DeepTrainer(net_arch)
conv_net.compile(optimizer=optimizer, lr=rate)

(None, 399)
(None, None)


In [5]:
conv_net.display_network_info()

Neural Network has 510699 trainable parameters
  #  Name            Shape       Parameters
---  --------------  --------  ------------
  0  aa_seq          1407x20              0
  1  Conv1           1407x300        300300
  2  maxpooling1d_1  1x300                0
  3  flatten_1       300                  0
  4  FCEnc1          300              90300
  5  Classifier      399             120099


In [ ]:
conv_net.fit(x_data, y_data,
             nb_epoch=num_epochs,
             batch_size=batch_size,
             validate=validate)

(5362, 1407, 20) (5362, 399)
Train on 5362 samples, validate on 1341 samples
Epoch 1/200
5362/5362 [==============================] - 8s - loss: 6.0359 - mean_cat_acc: 0.6255 - val_loss: 6.1054 - val_mean_cat_acc: 0.6219
Epoch 2/200
5362/5362 [==============================] - 8s - loss: 6.0116 - mean_cat_acc: 0.6270 - val_loss: 6.0880 - val_mean_cat_acc: 0.6227
Epoch 3/200
5362/5362 [==============================] - 8s - loss: 5.8658 - mean_cat_acc: 0.6361 - val_loss: 6.0209 - val_mean_cat_acc: 0.6264
Epoch 4/200
5362/5362 [==============================] - 8s - loss: 5.8348 - mean_cat_acc: 0.6380 - val_loss: 6.0157 - val_mean_cat_acc: 0.6264
Epoch 5/200
5362/5362 [==============================] - 8s - loss: 5.8038 - mean_cat_acc: 0.6402 - val_loss: 5.9937 - val_mean_cat_acc: 0.6286
Epoch 6/200
5362/5362 [==============================] - 8s - loss: 5.7958 - mean_cat_acc: 0.6404 - val_loss: 5.9821 - val_mean_cat_acc: 0.6294
Epoch 7/200
5362/5362 [==============================] - 8s

In [7]:
handle = Handle(batch_size=batch_size,
                filter_length=filter_length,
                filters=filters,
                program='CoMET',
                data_id=data_id)
handle.model = 'DeepCoFAM'
conv_net.save_train_history(handle)
conv_net.save_model_to_file(handle)

History saved to: models/dnabind/300_50_200_50_DeepCoFAM.history
Model saved to: models/dnabind/300_50_200_50_DeepCoFAM.model
